In [ ]:
import numpy as np
import scanpy as sc
import seaborn as sns
import os
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import sys
import pickle as pkl
path_helper = ["C:\\","Users","vfriedrich","projects","monkey_IZI","git_documentation","scRNAseq_cross_species_primate_human","analysis","helper"]
sys.path.append(os.path.join(*path_helper))
import helperVDF as h

import sccoda
from sccoda.util import comp_ana as mod
from sccoda.util import cell_composition_data as dat
from sccoda.util import data_visualization as viz
import sccoda.datasets as scd

In [ ]:
pre = "MH121"
drive = 'F'
base_model_path,base_table_path,base_plots_path,base_anndata_objects = h.return_local_paths(drive = drive,
                                                                                            pre = pre,
                                                                                            add_path = True)

def rel_change(base_val,new_val):
    return (new_val -base_val)/base_val

def make_df_annotation_azimut1_5_scanvi_v2(df):
    df['cluster_azimut1_5_scanvi_v2'] = df['cluster_azimut1_5_scanvi'].replace(
    {'NK': 'NK+Proliferating', 'NK Proliferating': 'NK+Proliferating'})
    return df

In [ ]:
_,base_table_path_MH105,_,_ = h.return_local_paths(drive = drive,pre = 'MH105',add_path = False)

In [ ]:
path_clusteranno_s0140 = 'F:\\monkey_IZI\\analysis\\tables\\s0140_clusteranno_scvi.txt.gz'
clusteranno_s0140 = pd.read_csv(path_clusteranno_s0140,compression = 'gzip',index_col=0)

In [ ]:
species = 'human'
_,_,_,base_anndata_objects_H24 = h.return_local_paths(drive,pre='H24',add_path = False)
adata_human = sc.read_h5ad(os.path.join(base_anndata_objects_H24,'H24' + '_' + species + '_anno_celltypes_v0.h5ad'))
adata_human.obs = pd.merge(adata_human.obs, clusteranno_s0140, left_index=True, right_index=True)
adata_human.obs = make_df_annotation_azimut1_5_scanvi_v2(adata_human.obs)
adata_human.obs.rename(columns={"timepoint_x": "timepoint", "individual_x": "individual","species_x" : "species"}, inplace=True)

In [ ]:
for celltype in ['CD8 T','CD16 Mono','NK+Proliferating']:
    celltype_col = 'cluster_azimut1_5_scanvi_v2'
    adata_human_ct = adata_human[adata_human.obs[celltype_col] == celltype]
    adata_human_ct.obs['scvi_clusters'] = adata_human_ct.obs['scvi_clusters'].astype('category')
    
    df_human_ct =  adata_human_ct.obs[[celltype_col,'timepoint','scvi_clusters']]
    all_timepoints = ['00hr', '06hr', '24hr']
    all_clusters = df_human_ct['scvi_clusters'].unique()
    all_combinations = pd.MultiIndex.from_product([all_timepoints, all_clusters], names=['timepoint', 'scvi_clusters']).to_frame(index=False)
    cell_counts_human = df_human_ct.groupby(['timepoint','scvi_clusters']).size().reset_index(name='count')
    cell_counts_human = pd.merge(all_combinations, cell_counts_human, on=['timepoint', 'scvi_clusters'], how='left')
    cell_counts_human['count'].fillna(0, inplace=True)
    cell_counts_pivot_human = cell_counts_human.pivot(index='timepoint',columns = 'scvi_clusters',values = 'count').reset_index()
    
    scoda_obj_human = dat.from_scanpy(
        adata_human_ct,
        cell_type_identifier='scvi_clusters',
        sample_identifier="timepoint",
        )
    
    #save_path_human_h5ad = os.path.join(base_anndata_objects,pre + '_scoda_obj_human.h5ad')
    #scoda_obj_human.write_h5ad(save_path_human_h5ad)
    
    df_all_human = pd.DataFrame(scoda_obj_human.X,columns = scoda_obj_human.var.index,index = scoda_obj_human.obs.index)
    proportions = df_all_human.div(df_all_human.sum(axis=0), axis=1)
    proportions = proportions.T
    
    proportions.to_csv(os.path.join(base_table_path,pre + '_' + str(celltype) +  '_proportions_human.csv'))
    

    scoda_obj_human.obs['Condition'] = scoda_obj_human.obs.index 
    scoda_model = mod.CompositionalAnalysis(scoda_obj_human, formula="Condition", reference_cell_type="automatic")
    
    
    sim_results = scoda_model.sample_hmc()
    
    sim_results.summary() 
    
    sim_results.set_fdr(est_fdr=0.25)
    print(sim_results.summary())
    
    print(sim_results.credible_effects())
    
    
    # saving
    save_path = os.path.join(base_table_path,pre + '_' + str(celltype) + '_scoda_sim_results_human.pkl')
    sim_results.save(save_path)
    

In [ ]:
species = 'cyno'
_,_,_,base_anndata_objects_M24 = h.return_local_paths(drive,pre='M24',add_path = False)
adata_cyno = sc.read_h5ad(os.path.join(base_anndata_objects_M24,'M24' + '_' + species + '_anno_celltypes_v0.h5ad'))
adata_cyno.obs = pd.merge(adata_cyno.obs, clusteranno_s0140, left_index=True, right_index=True)
adata_cyno.obs = make_df_annotation_azimut1_5_scanvi_v2(adata_cyno.obs)
adata_cyno.obs.rename(columns={"timepoint_x": "timepoint", "individual_x": "individual","species_x" : "species"}, inplace=True)

In [ ]:
for celltype in ['CD8 T','CD16 Mono','NK+Proliferating']:
    adata_cyno_ct = adata_cyno[adata_cyno.obs[celltype_col] == celltype]
    adata_cyno_ct.obs['scvi_clusters'] = adata_cyno_ct.obs['scvi_clusters'].astype('category')
    
    df_cyno_ct =  adata_cyno_ct.obs[[celltype_col,'timepoint','scvi_clusters']]
    all_timepoints = ['00hr', '06hr', '24hr']
    all_clusters = df_human_ct['scvi_clusters'].unique()
    all_combinations = pd.MultiIndex.from_product([all_timepoints, all_clusters], names=['timepoint', 'scvi_clusters']).to_frame(index=False)
    cell_counts_cyno = df_cyno_ct.groupby(['timepoint','scvi_clusters']).size().reset_index(name='count')
    cell_counts_cyno= pd.merge(all_combinations, cell_counts_cyno, on=['timepoint', 'scvi_clusters'], how='left')
    cell_counts_human['count'].fillna(0, inplace=True)
    cell_counts_pivot_cyno = cell_counts_cyno.pivot(index='timepoint',columns = 'scvi_clusters',values = 'count').reset_index()
    
    scoda_obj_cyno = dat.from_scanpy(
        adata_cyno_ct,
        cell_type_identifier='scvi_clusters',
        sample_identifier="timepoint",
        )
    
    #save_path_cyno_h5ad = os.path.join(base_anndata_objects,pre + '_scoda_obj_cyno.h5ad')
    #scoda_obj_cyno.write_h5ad(save_path_cyno_h5ad)
    
    df_all_cyno = pd.DataFrame(scoda_obj_cyno.X,columns = scoda_obj_cyno.var.index,index = scoda_obj_cyno.obs.index)
    proportions = df_all_cyno.div(df_all_cyno.sum(axis=0), axis=1)
    proportions = proportions.T
    
    proportions.to_csv(os.path.join(base_table_path,pre + '_' + str(celltype) +  '_proportions_cyno.csv'))

    scoda_obj_cyno.obs['Condition'] = scoda_obj_cyno.obs.index 
    if celltype == 'NK+Proliferating':
        scoda_model = mod.CompositionalAnalysis(scoda_obj_cyno, formula="Condition", reference_cell_type='23')
    else:
        scoda_model = mod.CompositionalAnalysis(scoda_obj_cyno, formula="Condition", reference_cell_type="automatic")
    
    sim_results = scoda_model.sample_hmc()
    
    sim_results.summary() 
    
    sim_results.set_fdr(est_fdr=0.25)
    print(sim_results.summary())
    
    print(sim_results.credible_effects())
    
    # saving
    save_path = os.path.join(base_table_path,pre + '_' + str(celltype) + '_scoda_sim_results_cyno.pkl')
    sim_results.save(save_path)

## Human

In [ ]:
for celltype in ['CD8 T']:
    proportions_human = pd.read_csv(os.path.join(base_table_path,pre + '_' + str(celltype) +  '_proportions_human.csv'),index_col=0)
    proportions_human.index.name = 'Cell Type'
    proportions_human.reset_index(inplace=True)
    proportions_human['Cell Type'] = proportions_human['Cell Type'].astype(int).astype('category') 
    
    
    proportions_human.index = pd.CategoricalIndex(proportions_human.index)
    proportions_human['rel06hr'] = proportions_human.apply(lambda row: rel_change(row['00hr'], row['06hr']), axis=1)
    proportions_human['rel24hr'] = proportions_human.apply(lambda row: rel_change(row['00hr'], row['24hr']), axis=1)
    load_path_human = os.path.join(base_table_path,pre + '_' + str(celltype) + '_scoda_sim_results_human.pkl')
    # loading
    with open(load_path_human, "rb") as f:
        sim_results_loaded = pkl.load(f)
    
    cred_effects = sim_results_loaded.credible_effects()
    
    cred_effects_df = pd.DataFrame(cred_effects).reset_index()
    cred_effects_df['Cell Type'] = cred_effects_df['Cell Type'].astype(int).astype('category') 
    cred_effects_df06hr = cred_effects_df[cred_effects_df['Covariate'] == 'Condition[T.06hr]']
    cred_effects_df06hr = cred_effects_df06hr.rename(columns={'Final Parameter': 'cred_effect_06hr'})
    
    cred_effects_df24hr = cred_effects_df[cred_effects_df['Covariate'] == 'Condition[T.24hr]']
    cred_effects_df24hr = cred_effects_df24hr.rename(columns={'Final Parameter': 'cred_effect_24hr'})
    
    
    proportions_human_with_cred = pd.merge(cred_effects_df06hr[['Cell Type','cred_effect_06hr']], proportions_human, left_on='Cell Type', right_on='Cell Type')
    
    proportions_human_with_cred =  pd.merge(cred_effects_df24hr[['Cell Type','cred_effect_24hr']], proportions_human_with_cred, left_on='Cell Type',right_on='Cell Type')
    
    proportions_human_with_cred.index = proportions_human_with_cred['Cell Type']
    proportions_human_with_cred.to_csv(os.path.join(base_table_path,pre + '_proportions_human_with_cred_' + str(celltype) +'.csv'))
    
    
    color_timepoints = ["#3b7c70", "#ce9642", "#3b3a3e"]
    proportions_human_with_cred[['00hr', '06hr', '24hr']].plot(kind='bar', figsize=(10, 4), width=0.9, color=color_timepoints)  # Increased width to 0.9
    plt.title('celltype proportions - human - ' +str(celltype))
    plt.xlabel('Cell type')
    plt.ylabel('Proportion')
    plt.xticks(rotation=0)
    plt.ylim([0, 1.05])
    
    n_groups = len(proportions_human_with_cred)
    n_bars_per_group = 3  
    bar_width = 0.9 / n_bars_per_group
    # Adding stars based on cred_effect conditions
    for i, (idx, row) in enumerate(proportions_human_with_cred.iterrows()):
        # Positions of the 06hr and 24hr bars within the group
        position_06hr = i  
        position_24hr = i + bar_width 
        
        offset = -0.02  # You can adjust this value as needed
        
    
        if row['cred_effect_06hr']:  # Check if True for 06hr
            plt.text(position_06hr , row['06hr'] + offset, '*', color='black', ha='center', va='bottom', fontsize=20)
            if row['rel06hr'] > 0:
                plt.text(position_06hr + 0.015, row['06hr'] + 0.08, r"$\mathbf{+}$" + f"{row['rel06hr'] * 100:.0f}%", color="#ce9642", ha='center', va='bottom', fontsize=10, rotation=90)
            else:
                plt.text(position_06hr + 0.015, row['06hr'] + 0.08, r"$\mathbf{-}$" + f"{-1*row['rel06hr'] * 100:.0f}%", color="#ce9642", ha='center', va='bottom', fontsize=10, rotation=90)
        if row['cred_effect_24hr']:  # Check if True for 24hr
            plt.text(position_24hr, row['24hr'] + offset, '*', color='black', ha='center', va='bottom', fontsize=20)
            if row['rel24hr'] > 0:
                plt.text(position_24hr + 0.015, row['24hr'] + 0.08, r"$\mathbf{+}$" + f"{row['rel24hr'] * 100:.0f}%", color="#3b3a3e", ha='center', va='bottom', fontsize=10, rotation=90)
            else:
                plt.text(position_24hr + 0.015, row['24hr'] + 0.08, r"$\mathbf{-}$" + f"{-1*row['rel24hr'] * 100:.0f}%", color="#3b3a3e", ha='center', va='bottom', fontsize=10, rotation=90)
    plt.savefig(os.path.join(base_plots_path,pre + '_barplot_proportion_human_' +str(celltype) + '.pdf'),bbox_inches='tight')

In [ ]:
proportions_human = pd.read_csv(os.path.join(base_table_path,pre + '_' + str(celltype) +  '_proportions_human.csv'),index_col=0)

In [ ]:
proportions_human['24hr'] = 0.0
proportions_human

In [ ]:
for celltype in ['CD16 Mono']:
    proportions_human = pd.read_csv(os.path.join(base_table_path,pre + '_' + str(celltype) +  '_proportions_human.csv'),index_col=0)
    proportions_human.index.name = 'Cell Type'
    proportions_human.reset_index(inplace=True)
    proportions_human['24hr'] = 0.0
    proportions_human['Cell Type'] = proportions_human['Cell Type'].astype(int).astype('category') 
    
    
    proportions_human.index = pd.CategoricalIndex(proportions_human.index)
    proportions_human['rel06hr'] = proportions_human.apply(lambda row: rel_change(row['00hr'], row['06hr']), axis=1)
    proportions_human['rel24hr'] = proportions_human.apply(lambda row: rel_change(row['00hr'], row['24hr']), axis=1)
    load_path_human = os.path.join(base_table_path,pre + '_' + str(celltype) + '_scoda_sim_results_human.pkl')
    # loading
    with open(load_path_human, "rb") as f:
        sim_results_loaded = pkl.load(f)
    
    cred_effects = sim_results_loaded.credible_effects()
    
    cred_effects_df = pd.DataFrame(cred_effects).reset_index()
    cred_effects_df['Cell Type'] = cred_effects_df['Cell Type'].astype(int).astype('category') 
    cred_effects_df06hr = cred_effects_df[cred_effects_df['Covariate'] == 'Condition[T.06hr]']
    cred_effects_df06hr = cred_effects_df06hr.rename(columns={'Final Parameter': 'cred_effect_06hr'})
    
    cred_effects_df24hr = cred_effects_df[cred_effects_df['Covariate'] == 'Condition[T.24hr]']
    cred_effects_df24hr = cred_effects_df24hr.rename(columns={'Final Parameter': 'cred_effect_24hr'})
    
    
    proportions_human_with_cred = pd.merge(cred_effects_df06hr[['Cell Type','cred_effect_06hr']], proportions_human, left_on='Cell Type', right_on='Cell Type')
    
    #proportions_human_with_cred =  pd.merge(cred_effects_df24hr[['Cell Type','cred_effect_24hr']], proportions_human_with_cred, left_on='Cell Type',right_on='Cell Type')
    
    proportions_human_with_cred.index = proportions_human_with_cred['Cell Type']
    proportions_human_with_cred.to_csv(os.path.join(base_table_path,pre + '_proportions_human_with_cred_' + str(celltype) +'.csv'))
    
    
    color_timepoints = ["#3b7c70", "#ce9642", "#3b3a3e"]
    proportions_human_with_cred[['00hr', '06hr', '24hr']].plot(kind='bar', figsize=(10, 4), width=0.9, color=color_timepoints)  # Increased width to 0.9
    plt.title('celltype proportions - human - ' +str(celltype))
    plt.xlabel('Cell type')
    plt.ylabel('Proportion')
    plt.xticks(rotation=90)
    plt.ylim([0, 1.05])
    
    n_groups = len(proportions_human_with_cred)
    n_bars_per_group = 3  
    bar_width = 0.9 / n_bars_per_group
    # Adding stars based on cred_effect conditions
    for i, (idx, row) in enumerate(proportions_human_with_cred.iterrows()):
        # Positions of the 06hr and 24hr bars within the group
        position_06hr = i  
        position_24hr = i + bar_width 
        
        offset = -0.02  # You can adjust this value as needed
        
    
        if row['cred_effect_06hr']:  # Check if True for 06hr
            plt.text(position_06hr , row['06hr'] + offset, '*', color='black', ha='center', va='bottom', fontsize=20)
            if row['rel06hr'] > 0:
                plt.text(position_06hr + 0.015, row['06hr'] + 0.08, r"$\mathbf{+}$" + f"{row['rel06hr'] * 100:.0f}%", color="#ce9642", ha='center', va='bottom', fontsize=10, rotation=90)
            else:
                plt.text(position_06hr + 0.015, row['06hr'] + 0.08, r"$\mathbf{-}$" + f"{-1*row['rel06hr'] * 100:.0f}%", color="#ce9642", ha='center', va='bottom', fontsize=10, rotation=90)
        
    plt.savefig(os.path.join(base_plots_path,pre + '_barplot_proportion_human_' +str(celltype) + '.pdf'),bbox_inches='tight')

In [ ]:
for celltype in ['NK+Proliferating']:
    proportions_human = pd.read_csv(os.path.join(base_table_path,pre + '_' + str(celltype) +  '_proportions_human.csv'),index_col=0)
    proportions_human.index.name = 'Cell Type'
    proportions_human.reset_index(inplace=True)
    proportions_human['Cell Type'] = proportions_human['Cell Type'].astype(int).astype('category') 
    
    
    proportions_human.index = pd.CategoricalIndex(proportions_human.index)
    proportions_human['rel06hr'] = proportions_human.apply(lambda row: rel_change(row['00hr'], row['06hr']), axis=1)
    proportions_human['rel24hr'] = proportions_human.apply(lambda row: rel_change(row['00hr'], row['24hr']), axis=1)
    load_path_human = os.path.join(base_table_path,pre + '_' + str(celltype) + '_scoda_sim_results_human.pkl')
    # loading
    with open(load_path_human, "rb") as f:
        sim_results_loaded = pkl.load(f)
    
    cred_effects = sim_results_loaded.credible_effects()
    
    cred_effects_df = pd.DataFrame(cred_effects).reset_index()
    cred_effects_df['Cell Type'] = cred_effects_df['Cell Type'].astype(int).astype('category') 
    cred_effects_df06hr = cred_effects_df[cred_effects_df['Covariate'] == 'Condition[T.06hr]']
    cred_effects_df06hr = cred_effects_df06hr.rename(columns={'Final Parameter': 'cred_effect_06hr'})
    
    cred_effects_df24hr = cred_effects_df[cred_effects_df['Covariate'] == 'Condition[T.24hr]']
    cred_effects_df24hr = cred_effects_df24hr.rename(columns={'Final Parameter': 'cred_effect_24hr'})
    
    
    proportions_human_with_cred = pd.merge(cred_effects_df06hr[['Cell Type','cred_effect_06hr']], proportions_human, left_on='Cell Type', right_on='Cell Type')
    
    proportions_human_with_cred =  pd.merge(cred_effects_df24hr[['Cell Type','cred_effect_24hr']], proportions_human_with_cred, left_on='Cell Type',right_on='Cell Type')
    
    proportions_human_with_cred.index = proportions_human_with_cred['Cell Type']
    proportions_human_with_cred.to_csv(os.path.join(base_table_path,pre + '_proportions_human_with_cred_' + str(celltype) +'.csv'))
    
    
    color_timepoints = ["#3b7c70", "#ce9642", "#3b3a3e"]
    proportions_human_with_cred[['00hr', '06hr', '24hr']].plot(kind='bar', figsize=(10, 4), width=0.9, color=color_timepoints)  # Increased width to 0.9
    plt.title('celltype proportions - human - ' +str(celltype))
    plt.xlabel('Cell type')
    plt.ylabel('Proportion')
    plt.xticks(rotation=0)
    plt.ylim([0, 1.05])
    
    n_groups = len(proportions_human_with_cred)
    n_bars_per_group = 3  
    bar_width = 0.9 / n_bars_per_group
    # Adding stars based on cred_effect conditions
    for i, (idx, row) in enumerate(proportions_human_with_cred.iterrows()):
        # Positions of the 06hr and 24hr bars within the group
        position_06hr = i  
        position_24hr = i + bar_width 
        
        offset = -0.02  # You can adjust this value as needed
        
    
        if row['cred_effect_06hr']:  # Check if True for 06hr
            plt.text(position_06hr , row['06hr'] + offset, '*', color='black', ha='center', va='bottom', fontsize=20)
            if row['rel06hr'] > 0:
                plt.text(position_06hr + 0.015, row['06hr'] + 0.08, r"$\mathbf{+}$" + f"{row['rel06hr'] * 100:.0f}%", color="#ce9642", ha='center', va='bottom', fontsize=10, rotation=90)
            else:
                plt.text(position_06hr + 0.015, row['06hr'] + 0.08, r"$\mathbf{-}$" + f"{-1*row['rel06hr'] * 100:.0f}%", color="#ce9642", ha='center', va='bottom', fontsize=10, rotation=90)
        if row['cred_effect_24hr']:  # Check if True for 24hr
            plt.text(position_24hr, row['24hr'] + offset, '*', color='black', ha='center', va='bottom', fontsize=20)
            if row['rel24hr'] > 0:
                plt.text(position_24hr + 0.015, row['24hr'] + 0.08, r"$\mathbf{+}$" + f"{row['rel24hr'] * 100:.0f}%", color="#3b3a3e", ha='center', va='bottom', fontsize=10, rotation=90)
            else:
                plt.text(position_24hr + 0.015, row['24hr'] + 0.08, r"$\mathbf{-}$" + f"{-1*row['rel24hr'] * 100:.0f}%", color="#3b3a3e", ha='center', va='bottom', fontsize=10, rotation=90)
    plt.savefig(os.path.join(base_plots_path,pre + '_barplot_proportion_human_' +str(celltype) + '.pdf'),bbox_inches='tight')

In [ ]:
color_timepoints = ["#3b7c70", "#ce9642", "#3b3a3e"]
proportions_human_with_cred[['00hr', '06hr', '24hr']].plot(kind='bar', figsize=(10, 4), width=0.9, color=color_timepoints)  # Increased width to 0.9
plt.title('celltype proportions - human')
plt.xlabel('Cell type')
plt.ylabel('Proportion')
plt.xticks(rotation=90)
plt.ylim([0, 0.87])

n_groups = len(proportions_human_with_cred)
n_bars_per_group = 3  
bar_width = 0.9 / n_bars_per_group
# Adding stars based on cred_effect conditions
for i, (idx, row) in enumerate(proportions_human_with_cred.iterrows()):
    # Positions of the 06hr and 24hr bars within the group
    position_06hr = i  
    position_24hr = i + bar_width 
    
    offset = -0.02  # You can adjust this value as needed
    

    if row['cred_effect_06hr']:  # Check if True for 06hr
        plt.text(position_06hr , row['06hr'] + offset, '*', color='black', ha='center', va='bottom', fontsize=20)
        if row['rel06hr'] > 0:
            plt.text(position_06hr + 0.015, row['06hr'] + 0.07, r"$\mathbf{+}$" + f"{row['rel06hr'] * 100:.0f}%", color="#ce9642", ha='center', va='bottom', fontsize=10, rotation=90)
        else:
            plt.text(position_06hr + 0.015, row['06hr'] + 0.07, r"$\mathbf{-}$" + f"{-1*row['rel06hr'] * 100:.0f}%", color="#ce9642", ha='center', va='bottom', fontsize=10, rotation=90)
    if row['cred_effect_24hr']:  # Check if True for 24hr
        plt.text(position_24hr, row['24hr'] + offset, '*', color='black', ha='center', va='bottom', fontsize=20)
        if row['rel24hr'] > 0:
            plt.text(position_24hr + 0.015, row['24hr'] + 0.07, r"$\mathbf{+}$" + f"{row['rel24hr'] * 100:.0f}%", color="#3b3a3e", ha='center', va='bottom', fontsize=10, rotation=90)
        else:
            plt.text(position_24hr + 0.015, row['24hr'] + 0.07, r"$\mathbf{-}$" + f"{-1*row['rel24hr'] * 100:.0f}%", color="#3b3a3e", ha='center', va='bottom', fontsize=10, rotation=90)
plt.savefig(os.path.join(base_plots_path,pre + '_barplot_proportion_human.pdf'),bbox_inches='tight')


## Cyno

In [ ]:
for celltype in ['CD8 T','CD16 Mono','NK+Proliferating']:
    proportions_cyno = pd.read_csv(os.path.join(base_table_path,pre + '_' + str(celltype) +  '_proportions_cyno.csv'),index_col=0)
    proportions_cyno.index.name = 'Cell Type'
    proportions_cyno.reset_index(inplace=True)
    proportions_cyno['Cell Type'] = proportions_cyno['Cell Type'].astype(int).astype('category') 
    
    
    proportions_cyno.index = pd.CategoricalIndex(proportions_cyno.index)
    proportions_cyno['rel06hr'] = proportions_cyno.apply(lambda row: rel_change(row['00hr'], row['06hr']), axis=1)
    proportions_cyno['rel24hr'] = proportions_cyno.apply(lambda row: rel_change(row['00hr'], row['24hr']), axis=1)
    load_path_cyno = os.path.join(base_table_path,pre + '_' + str(celltype) + '_scoda_sim_results_cyno.pkl')
    # loading
    with open(load_path_cyno, "rb") as f:
        sim_results_loaded = pkl.load(f)
    
    cred_effects = sim_results_loaded.credible_effects()
    
    cred_effects_df = pd.DataFrame(cred_effects).reset_index()
    cred_effects_df['Cell Type'] = cred_effects_df['Cell Type'].astype(int).astype('category') 
    cred_effects_df06hr = cred_effects_df[cred_effects_df['Covariate'] == 'Condition[T.06hr]']
    cred_effects_df06hr = cred_effects_df06hr.rename(columns={'Final Parameter': 'cred_effect_06hr'})
    
    cred_effects_df24hr = cred_effects_df[cred_effects_df['Covariate'] == 'Condition[T.24hr]']
    cred_effects_df24hr = cred_effects_df24hr.rename(columns={'Final Parameter': 'cred_effect_24hr'})
    
    
    proportions_cyno_with_cred = pd.merge(cred_effects_df06hr[['Cell Type','cred_effect_06hr']], proportions_cyno, left_on='Cell Type', right_on='Cell Type')
    
    proportions_cyno_with_cred =  pd.merge(cred_effects_df24hr[['Cell Type','cred_effect_24hr']], proportions_cyno_with_cred, left_on='Cell Type',right_on='Cell Type')
    
    proportions_cyno_with_cred.index = proportions_cyno_with_cred['Cell Type']
    proportions_cyno_with_cred.to_csv(os.path.join(base_table_path,pre + '_proportions_cyno_with_cred_' + str(celltype) +'.csv'))
    
    
    color_timepoints = ["#3b7c70", "#ce9642", "#3b3a3e"]
    proportions_cyno_with_cred[['00hr', '06hr', '24hr']].plot(kind='bar', figsize=(10, 4), width=0.9, color=color_timepoints)  # Increased width to 0.9
    plt.title('celltype proportions - cyno - ' +str(celltype))
    plt.xlabel('Cell type')
    plt.ylabel('Proportion')
    plt.xticks(rotation=0)
    plt.ylim([0, 1.05])
    
    n_groups = len(proportions_cyno_with_cred)
    n_bars_per_group = 3  
    bar_width = 0.9 / n_bars_per_group
    # Adding stars based on cred_effect conditions
    for i, (idx, row) in enumerate(proportions_cyno_with_cred.iterrows()):
        # Positions of the 06hr and 24hr bars within the group
        position_06hr = i  
        position_24hr = i + bar_width 
        
        offset = -0.02  # You can adjust this value as needed
        
    
        if row['cred_effect_06hr']:  # Check if True for 06hr
            plt.text(position_06hr , row['06hr'] + offset, '*', color='black', ha='center', va='bottom', fontsize=20)
            if row['rel06hr'] > 0:
                plt.text(position_06hr + 0.015, row['06hr'] + 0.08, r"$\mathbf{+}$" + f"{row['rel06hr'] * 100:.0f}%", color="#ce9642", ha='center', va='bottom', fontsize=10, rotation=90)
            else:
                plt.text(position_06hr + 0.015, row['06hr'] + 0.08, r"$\mathbf{-}$" + f"{-1*row['rel06hr'] * 100:.0f}%", color="#ce9642", ha='center', va='bottom', fontsize=10, rotation=90)
        if row['cred_effect_24hr']:  # Check if True for 24hr
            plt.text(position_24hr, row['24hr'] + offset, '*', color='black', ha='center', va='bottom', fontsize=20)
            if row['rel24hr'] > 0:
                plt.text(position_24hr + 0.015, row['24hr'] + 0.08, r"$\mathbf{+}$" + f"{row['rel24hr'] * 100:.0f}%", color="#3b3a3e", ha='center', va='bottom', fontsize=10, rotation=90)
            else:
                plt.text(position_24hr + 0.015, row['24hr'] + 0.08, r"$\mathbf{-}$" + f"{-1*row['rel24hr'] * 100:.0f}%", color="#3b3a3e", ha='center', va='bottom', fontsize=10, rotation=90)
    plt.savefig(os.path.join(base_plots_path,pre + '_barplot_proportion_cyno_' +str(celltype) + '.pdf'),bbox_inches='tight')

## Save session

In [ ]:
base_package_version_path = h.return_package_version_local_path(drive=drive)
h.save_package_versions(base_package_version_path,pre,do_print = True)
h.print_main_versions()